In [ ]:
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random
import gc

In [ ]:
# загрузка исходного изображения и маски
img = cv2.imread('PATH_to_IMAGE')
mask = cv2.imread('PATH_to_MASK')

In [ ]:
# фрагментация изображения и маски
height, width, channels = img.shape
cropped_img = img[0:(height // 256), 0:(width // 256)]
cropped_mask = mask[0:(height // 256), 0:(width // 256)]
img = cropped_img
img2 = img
mask = cropped_mask
mask2 = mask
height, width, channels = img.shape
W_SIZE  = width // 256
H_SIZE = height // 256

images = []
segms = []
for ih in range(H_SIZE ):
   for iw in range(W_SIZE ):
      x = width/W_SIZE * iw
      y = height/H_SIZE * ih
      h = (height / H_SIZE)
      w = (width / W_SIZE )
      img = img[int(y):int(y+h), int(x):int(x+w)]
      mask = mask[int(y):int(y+h), int(x):int(x+w)]
      images.append(img)
      segms.append(mask)
      img = img2

In [ ]:
# аугментация
images_aug = []
segms_aug = []
for i in range(len(images)):
  aug_1 = A.HorizontalFlip(p=1)
  augmented_1 = aug_1(image=images[i], mask=segms[i])
  images_aug.append(images_aug, augmented_1['image'])
  segms_aug.append(segms_aug, augmented_1['mask'])
  aug_2 = A.VerticalFlip(p=1)
  augmented_2 = aug_2(image=images[i], mask=segms[i])
  images_aug.append(images_aug, augmented_2['image'])
  segms_aug.append(segms_aug, augmented_2['mask'])
  aug_3 = A.RandomRotate90(p=1)
  augmented_3 = aug_3(image=images[i], mask=segms[i])
  images_aug.append(images_aug, augmented_3['image'])
  segms_aug.append(segms_aug, augmented_3['mask'])
  aug_4 = A.Transpose(p=1)
  augmented_4 = aug_4(image=images[i], mask=segms[i])
  images_aug.append(images_aug, augmented_4['image'])
  segms_aug.append(segms_aug, augmented_4['mask'])
  aug_5 = A.GridDistortion(p=1)
  random.seed(7)
  augmented_5 = aug_5(image=images[i], mask=segms[i])
  images_aug.append(images_aug, augmented_5['image'])
  segms_aug.append(segms_aug, augmented_5['mask'])

In [ ]:
data = np.concatenate((np.array(images), np.array(images_aug)), axis=0)
masks = np.concatenate((np.array(segms), np.array(segms_aug)), axis=0)

In [ ]:
# перевод масок в категориальный вид
def image_cat(image, black_color = 200):
  pic = np.array(image)
  img = np.zeros((pic.shape[0], pic.shape[1], 1))
  np.place(img[ :, :, 0], (pic[ :, :, 0] + pic[ :, :, 1] + pic[ :, :, 2]) < black_color, 0)
  np.place(img[ :, :, 0], (pic[ :, :, 0] + pic[ :, :, 1] + pic[ :, :, 2]) >= black_color, 1)
  del pic
  gc.collect()
  return img

def images_cat(images):
  newImages = []
  for t in range(images.shape[0]):
    img = images[t].copy()
    new_img = image_cat(img)
    new_img = np.array(new_img, int)
    newImages.append(new_img)
    del img, new_img
    gc.collect()
  newImages = np.array(newImages)
  return newImages

masks = images_cat(masks).squeeze(3)

In [ ]:
# нормализация изображений
t1 = A.Compose([
    A.Resize(256,256),
    A.augmentations.transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])

datas = []
for i in range(len(data)):
  aug = t1(image=data[i])
  datas.append(aug['image'])
datas = np.array(datas)
del aug, data
gc.collect()